In [9]:
import requests
import requests_oauthlib
import json
import urllib3
from typing import Any

requests.packages.urllib3.disable_warnings()

<h1>Table of Contents</h1>

- <p style="font-size:15pt;">Availability Check</p>
- <p style="font-size:15pt;">Conncet Device</p>
- <p style="font-size:15pt;">Register Callbacks</p>
- <p style="font-size:15pt;">UI Configuration</p>
- <p style="font-size:15pt;">Values</p>



<h2>Note</h2>

- <p style="font-size:12pt;">I will not build in Value Checking since the functions are shown and should be clear</p>


____________________

<h2>Availability Check</h2>

It is possible to check if the Gira IoT Rest API is available. Without prior resgistration or authentication

soley send a GET request with 'api/v2' to the ip of your X1

IF the response is 200 the API is available


if it does not work with https: try it with http:

In [11]:
ip = '10.220.9.114'
url = f'https://{ip}/api/v2'

In [15]:
response = requests.get(url,verify=False)

In [16]:
response

<Response [200]>

________________

<h2>Connect Device</h2>

"All Gira IoT REST API access is via HTTPS. All data is sent and received as JSON."

"To use the API a client has to register with a username and password for authorization. 
The registration for the API is using HTTPS with basic access authentication only."
<h3>Important</h3>
<p style="color:Tomato;">The HTTPS connection is not fully trusted, because it is not technically possible that the server provides a trusted TLS certificate.
Because of this, the client that is used to access the Gira IoT REST API needs to skip the certification check. 
Please consult the documentation of the client software or library on how to skip the certification check.</p>


In [12]:
def connect (ip:str, username:str, password:str):

    try:
        requests.packages.urllib3.disable_warnings()
        ssl_certificate = 'Utils/Zertifikate2.cer'
        body = '{"client":"de.homeandfuture.FristTestClient"}'
        url = f'https://{ip}/api/v2/clients'
        response = requests.post(url, body, auth=(username, password), verify=False)
        
        return response
    except Exception as e:
        return e

In [13]:
response_connect = connect('10.220.9.114','Justus','1111')

In [14]:
response_connect.json()['token']

'tBSXdQtJwFrF88GzH9Tag32Xvi3FCYsv'

From now on my working token will be

<p style="color:Tomato;">tBSXdQtJwFrF88GzH9Tag32Xvi3FCYsv</p>

the user is 'Justus'

-------------------

<h2>Register Callbacks</h2>

If testCallbacks is set to true, the callback server must respond with 200 OK. Only one set of callback-URLs can be registered per < access token >.

<h3>Data</h3>
{

"serviceCallback": "callback URL of service events, optional",

"valueCallback": "callback URL of value events, optional",

"testCallbacks": "boolean value, optional"

}

In [8]:
def register_callbacks (ip:str, token:str, serviceCallback:str, valueCallback:str, testCallbacks:bool = None):

    if testCallbacks == None:
        data = {
                "serviceCallback": serviceCallback, 
                "valueCallback": valueCallback
                }
    else:
        data = {
                "serviceCallback": serviceCallback, 
                "valueCallback": valueCallback, 
                "testCallbacks": testCallbacks
                }

    try:
        url = f'https://{ip}/api/v2/clients/{token}/callbacks'
        response = requests.post(url, verify=False)
        return response
    except Exception as e:
        return e

In [9]:
serviceCallback = 'Test'
valueCallback = 'Test'
testCallbacks = None

response_callback = register_callbacks('10.220.9.114','tBSXdQtJwFrF88GzH9Tag32Xvi3FCYsv', serviceCallback, valueCallback, testCallbacks)

In [10]:
response_callback.json()

{'error': {'code': 'missingContent', 'message': 'Missing content.'}}

<h3>Remember</h3>

To use callbacks, a separate HTTPS-Server has to be used, where the REST API can send the callback events to.


_____________________

<h2>UI configuration</h2>

This Block consists of 2 Parts 
1. Get the UID
2. Get the configuration

<h3>Get the UID</h3>

GET /api/uiconfig/uid

Unique identifier of current configuration. This identifier changes each time the configuration is changed (e.g. GPA project download, configuration changes with the Gira Smart Home App).

In [7]:
def get_config_uid (ip:str, token:str):

    try:
        url = f'https://{ip}/api/v2/uiconfig/uid?token={token}'
        response = requests.get(url, verify=False)
        return response
    except Exception as e:
        return e 

In [15]:
response_uid = get_config_uid('10.220.9.114','tBSXdQtJwFrF88GzH9Tag32Xvi3FCYsv')

In [17]:
response_uid.json(), response_uid

({'uid': 'a0e3'}, '{"uid":"a0e3"}')

From now on my working uid will be <p style="color:Tomato;">a0e3</p>

_____________________

<h3>Get the configuration</h3>

GET /api/uiconfig[?expand=dataPointFlags,parameters,locations,trades]

Response
Get complete UI configuration.
- uid - The unique UI configuration identifier.
- functionType - Function types unique resource name. See 9.1 Function definitions for further information.
- channelType - Channel types unique resource name. See 9.2 Channel definitions for further information.
- displayName - UTF-8 based display name.
- functions - A list of all functions.
- dataPoints - A list of all available data points in the function.
- uid - The unique identifier of the data point.
- name - The logical name of the data point based on the channel definition.
- canRead - Whether the data point can be read.
Will be returned if dataPointFlags present within expand parameter only.
- canWrite - Whether the data point can be written.
Will be returned if dataPointFlags present within expand parameter only.
- canEvent - Whether the data point can event.
Will be returned if dataPointFlags present within expand parameter only.
- parameters - A list of function parameters.
Will be returned if present within expand parameter only.
- locations - A nested list of all locations and the contained unique function identifiers. Will be returned if present within expand parameter only.
- trades - A list of all trades and the contained unique function identifiers. Will be returned if present within expand parameter only.

In [85]:
def get_uid_config (ip:str, token:str, uid:str):
    try:
        url = f'https://{ip}/api/uiconfig?token={token}'
        response = requests.get(url, verify=False)
        return response
    except Exception as e:
        return e

In [86]:
uid_config = get_uid_config('10.220.9.114','1cUEe8F9ZZGQvIMvR7jpm6WP4IJbB1TT', 'a0e3')

In [95]:
uid_config

<Response [200]>

<p style="font-size:12pt;">Now we have the whole current configuration</p>
<p style="font-size:12pt;">Let's save it to a .json file to have a look at it</p>

In [101]:
with open('uid_config.json', mode = 'w') as f:
    json.dump(uid_config.json(),f, indent=6)

In [100]:
functions = uid_config.json()['functions']
for function in functions:
    print(f"{function['displayName']}")

Sonos Hobbyraum
Hobbyraum Steckdose Nord-West
Hobbyraum Steckdose Süd-West
Fernzugriff
SD karten Speicherzustand
Datenlogger Status
Staubsauger
Deckenlich Waschküche
Deckenlich Carportkeller
Deckenlicht Kellerflur
Deckenlicht Basteleck
Keller Licht Zentral
Deckenlicht Fitnessraum
Sonos Wohnzimmer
Stehlampe Wohnzimmer
Steckdose Whonzimmer Links
Deckenlicht Wohnzimmer Nord alle
Deckenlicht Wohnzimmer Süd alle
Deckenlicht Wohnzimmer alle
Jalousie Wohnzimmer Süd
Jalousie Wohnzimmer Süd West Fest
Jalousie Wohnzimmer Süd-West Tür
Jalousie Wohnzimmer Nord-West Tür
Jalousie Wohnzimmer Nord-West Fest
Jalousie Wohnzimmer Nord
Deckenlicht Küche
Jalousie Küche
Sonos Küche
Deckenlicht Arbeitszimmer
Jalousie Arbeitszimmer Ost
Jalousie Arbeitszimmer Süd
Deckenlicht Diele
Jalousie Diele
Deckenlicht WC
Licht Erdgeschoss Zentral
Jalousie Erdgeschoss Zentral
Deckenlicht Justus
Jalousie Justus
Sonos Justus
Wandlicht Justus
Licht Justus
Licht Justus Aus
Sonos II
Deckenlicht Bad
Jalousie Bad
Sonos Bad
Decke

We now had a look at the current configuration of the X1 and understand it a little bit

________________________________________

<h2>Values</h2>

This Block consists of 3 Parts 
1. Get value(s)
2. Set value(s)
3. Set single value

<h3>Get value(s)</h3>

GET /api/values/{uid}

In [102]:
def get_uid_values (ip:str, token:str, uid:str):
    try:
        url = f'https://{ip}/api/values/{uid}?token={token}'
        response = requests.get(url, verify=False)
        return response
    except Exception as e:
        return e

In [105]:
uid_values = get_uid_values('10.220.9.114','1cUEe8F9ZZGQvIMvR7jpm6WP4IJbB1TT', 'a02m')

In [106]:
uid_values.json(),uid_values

({'values': [{'uid': 'a02n', 'value': '1'},
   {'uid': 'a02o', 'value': '-0'},
   {'uid': 'a02p', 'value': '7'}]},
 <Response [200]>)

As we can see for a normal light we get a list of the value dicts and those are matching with the ones from the uid config

<br>"dataPoints": [</br>
<br>            {
<br>                  "name": "OnOff",</br>
<br>                  "uid": "a02n"</br>
<br>            },</br>
<br>            {</br>
<br>                  "name": "Shift",</br>
<br>                  "uid": "a02o"</br>
<br>            },</br>
<br>            {</br>
<br>                  "name": "Brightness",</br>
<br>                  "uid": "a02p"</br>
<br>            }</br>


as we can conclude 
- "OnOff" has value | 1 | 
- "Shift" has value | -0 | 
- Brightness has value | 7 |

----------------------

<h3>Set value(s)</h3>

PUT /api/values

In [205]:
def set_values (ip:str, token:str, uid:str, value: Any):

    try:
        url = f'https://{ip}/api/v2/values?token={token}'
        body = {
                    "values": [
                        {
                            "uid": "a02n",
                            "value": value
                        }, 
                        {
                            "uid": "a02o",
                            "value": -0 
                        },
                        {
                            "uid": "a02p",
                            "value": 7
                        }

                    ] 
                }
        response = requests.put(url, json=body,verify=False)
        return response
    except Exception as e:
        return e

def set_value (ip:str, token:str, uid:str, value: Any):

    try:
        url = f'https://{ip}/api/values/{uid}?token={token}'
        body = { "value" : value }
        response = requests.put(url, json=body, verify=False)
        return response
    except Exception as e:
        return e

In [208]:
value_set = set_value('10.220.9.114','1cUEe8F9ZZGQvIMvR7jpm6WP4IJbB1TT', 'a02n', 0)

In [209]:
value_set

<Response [200]>

Important to notice is when a value set is succesfull no json is returned only a Response [200]

<p style="font-size:15pt;">WE DID IT</p>

<p style="font-size:12pt;">we successfully set a value</p>